In [ ]:
import physionet_challenge_utility_script as pc

import ecg_plot
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_addons as tfa
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.utils.vis_utils import plot_model
from keras.preprocessing.sequence import pad_sequences
%load_ext autoreload
%autoreload
%reload_ext autoreload

def plot_ecg(path):
    ecg_data = pc.load_challenge_data(path)
    ecg_plot.plot(ecg_data[0]/1000, sample_rate=500, title='')
    ecg_plot.show()
plot_ecg("SM22_MLData/China12Lead/Q0001.mat")

In [ ]:
def shuffle_batch_generator_demo(batch_size, gen_x,gen_y, gen_z): 
    np.random.shuffle(order_array)
    batch_features = np.zeros((batch_size,5000, 12))
    batch_labels = np.zeros((batch_size,snomed_classes.shape[0])) #drop undef class
    batch_demo_data = np.zeros((batch_size,2))
    while True:
        for i in range(batch_size):

            batch_features[i] = next(gen_x)
            batch_labels[i] = next(gen_y)
            batch_demo_data[i] = next(gen_z)

        X_combined = [batch_features, batch_demo_data]
        yield X_combined, batch_labels
        
def shuffle_batch_generator(batch_size, gen_x,gen_y): 
    np.random.shuffle(order_array)
    batch_features = np.zeros((batch_size,5000, 12))
    batch_labels = np.zeros((batch_size,snomed_classes.shape[0])) #drop undef class
    while True:
        for i in range(batch_size):

            batch_features[i] = next(gen_x)
            batch_labels[i] = next(gen_y)
            
        yield batch_features, batch_labels

def generate_y_shuffle(y_train):
    while True:
        for i in order_array:
            y_shuffled = y_train[i]
            yield y_shuffled


def generate_X_shuffle(X_train):
    while True:
        for i in order_array:
                #if filepath.endswith(".mat"):
                    data, header_data = pc.load_challenge_data(X_train[i])
                    X_train_new = pad_sequences(data, maxlen=5000, truncating='post',padding="post")
                    X_train_new = X_train_new.reshape(5000,12)
                    yield X_train_new

def generate_z_shuffle(age_train, gender_train):
    while True:
        for i in order_array:
            gen_age = age_train[i]
            gen_gender = gender_train[i]
            z_train = [gen_age , gen_gender]
            yield z_train

In [ ]:
DATADIR = "./SM22_MLData"

In [ ]:
gender, age, labels, ecg_filenames = pc.import_key_data(DATADIR)

age, gender = pc.import_gender_and_age(age, gender)

SNOMED_scored=pd.read_csv(f"{DATADIR}/SNOMEDMappings/SNOMED_mappings_scored.csv", sep=";")
SNOMED_unscored=pd.read_csv(f"{DATADIR}/SNOMEDMappings/SNOMED_mappings_unscored.csv", sep=";")
df_labels = pc.make_undefined_class(labels,SNOMED_unscored)

y , snomed_classes = pc.onehot_encode(df_labels)

snomed_abbr = []
for j in range(len(snomed_classes)):
    for i in range(len(SNOMED_scored.iloc[:,1])):
        if (str(SNOMED_scored.iloc[:,1][i]) == snomed_classes[j]):
            snomed_abbr.append(SNOMED_scored.iloc[:,2][i])
            
snomed_abbr = np.asarray(snomed_abbr)

y_all_comb = pc.get_labels_for_all_combinations(y)

folds = pc.split_data(labels, y_all_comb)

In [ ]:
firstModelData = {}
firstModelData["gender"], firstModelData["age"], firstModelData["labels"], firstModelData["ecg_filenames"] = pc.import_key_data(DATADIR)
firstModelData["ecg_filenames"] = np.asarray(firstModelData["ecg_filenames"])

print(len(firstModelData["labels"]))
for i in range(len(firstModelData["labels"])):
    splitLabels = firstModelData["labels"][i].split(",")
    isSR, isNSR, totalRhythm = 0, 0, 0
    if len(splitLabels) > 1:
        isNSR = 2 #NSR
    if not("426783006" in splitLabels):
        isNSR = 2 #NSR
    if "426783006" in splitLabels:
        isSR = 1 #SR
    totalRhythm = isSR + isNSR
    firstModelData["labels"][i] = str(totalRhythm)
    
firstModelData["labels"][0] = "2,undefined"

firstModelData["age"], firstModelData["gender"] = pc.import_gender_and_age(firstModelData["age"], firstModelData["gender"])

firstModelData["SNOMED_scored"] = pd.read_csv(f"{DATADIR}/SNOMEDMappings/SNOMED_mappings_scored.csv", sep=";")
firstModelData["SNOMED_unscored"] = pd.read_csv(f"{DATADIR}/SNOMEDMappings/FIRSTMODELMapping.csv", sep=",")
firstModelData["df_labels"] = pd.DataFrame(firstModelData["labels"])

firstModelData["y"], firstModelData["snomed_classes"] = pc.onehot_encode(firstModelData["df_labels"])

firstModelData["y_all_comb"] = pc.get_labels_for_all_combinations(firstModelData["y"])
firstModelData["folds"] = pc.split_data(firstModelData["labels"], firstModelData["y_all_comb"])

In [ ]:
secondModelData = {}
secondModelData["gender"], secondModelData["age"], secondModelData["labels"], secondModelData["ecg_filenames"] = pc.import_key_data(DATADIR)
secondModelData["ecg_filenames"] = np.asarray(firstModelData["ecg_filenames"])

for i in range(len(secondModelData["labels"])-1, -1, -1):
    splitLabel = secondModelData["labels"][i].split(",")
    if "426783006" in splitLabel:
        splitLabel.pop(splitLabel.index("426783006"))
        secondModelData["labels"][i] = ",".join(splitLabel)
            
ecg_filenames = np.asarray(ecg_filenames)

secondModelData["age"], secondModelData["gender"] = pc.import_gender_and_age(secondModelData["age"], secondModelData["gender"])

secondModelData["SNOMED_scored"] = pd.read_csv(f"{DATADIR}/SNOMEDMappings/SNOMED_mappings_scored.csv", sep=";")
secondModelData["SNOMED_unscored"] = pd.read_csv(f"{DATADIR}/SNOMEDMappings/SNOMED_mappings_unscored.csv", sep=";")
secondModelData["df_labels"] = pc.make_undefined_class(secondModelData["labels"],secondModelData["SNOMED_unscored"])

secondModelData["y"] , secondModelData["snomed_classes"] = pc.onehot_encode(secondModelData["df_labels"])

secondModelData["snomed_abbr"] = []
for j in range(len(secondModelData["snomed_classes"])):
    for i in range(len(secondModelData["SNOMED_scored"].iloc[:,1])):
        if (str(secondModelData["SNOMED_scored"].iloc[:,1][i]) == secondModelData["snomed_classes"][j]):
            secondModelData["snomed_abbr"].append(secondModelData["SNOMED_scored"].iloc[:,2][i])
            
secondModelData["snomed_abbr"] = np.asarray(secondModelData["snomed_abbr"])
secondModelData["y_all_comb"] = pc.get_labels_for_all_combinations(secondModelData["y"])
secondModelData["folds"] = pc.split_data(secondModelData["labels"], secondModelData["y_all_comb"])

In [ ]:
order_array = folds[0][0]

new_weights = pc.calculating_class_weights(y)

keys = np.arange(0,27,1)
weight_dictionary = dict(zip(keys, new_weights.T[1]))

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_AUC', factor=0.1, patience=3, verbose=1, mode='max',
    min_delta=0.0001, cooldown=0, min_lr=0
)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', mode='max', verbose=1, patience=2)

from scipy import optimize
def thr_chall_metrics(thr, label, output_prob):
    return -pc.compute_challenge_metric_for_opt(label, np.array(output_prob>thr))

In [ ]:
model_1 = pc.residual_network_1d(classes=3)
model.load_weights("secondModelWeights_30v1.h5")
model_2 = pc.FCN(classes=26)